## Домашняя работа

Взять boston house-prices datase (sklearn.datasets.load_boston). Возмите 7 любых регрессоров (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество).

In [12]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
boston = load_boston()

X = boston.data
y = boston.target

In [5]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,BaggingRegressor,GradientBoostingRegressor, RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet

from sklearn.model_selection import cross_val_score,RandomizedSearchCV
from sklearn.metrics import mean_squared_error

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3)

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [37]:
algoritms = [
    {'name': 'KNeighbors' , 'model': KNeighborsRegressor(n_jobs=-1)  , 'params': {'n_neighbors':list(range(3,23)),'weights' : ['uniform', 'distance']   }},
    {'name': 'BagReg' , 'model': BaggingRegressor(n_jobs=-1)   , 'params': { 'base_estimator':[RandomForestRegressor(),DecisionTreeRegressor(),SVR()],'n_estimators':list(range(3,23)) }},
    {'name': 'Aboost' , 'model': AdaBoostRegressor()  , 'params': {'base_estimator':[RandomForestRegressor(),DecisionTreeRegressor(),SVR()],'n_estimators':list(range(10,33)),'learning_rate':np.arange(1,5,.5),'loss': ['linear', 'square', 'exponential']     }},
    {'name': 'Gboost' , 'model': GradientBoostingRegressor()  , 'params': {'loss' : ['ls', 'lad', 'huber'],'learning_rate':np.arange(.1,2,.1),'n_estimators':list(range(100,500,1)),'criterion' : ['friedman_mse', 'mse', 'mae'],'ccp_alpha':np.arange(1,5,.5)      }},
    {'name': 'RandFrst' , 'model': RandomForestRegressor(n_jobs=-1)   , 'params': {'n_estimators':list(range(100,500,1)),'criterion': ["mse", "mae"],'max_depth':list(range(5,100))+[None],'ccp_alpha':np.arange(.1,2,.1)     }},
    {'name': 'LightGB' , 'model': LGBMRegressor(n_jobs=-1) , 'params': {'n_estimators':list(range(100,500,1)),'boosting_type':['gbdt','goss','rf'],'max_depth':list(range(5,100))+[None],'reg_alpha':np.arange(0,1,.1),'reg_lambda':np.arange(0,1,.1)      }},
    {'name': 'ElastNt' , 'model': ElasticNet()   , 'params': {'alpha': np.arange(1,5,.5), 'l1_ratio':np.arange(0,1.1,.1)   }}
]

In [ ]:
results = []

for algo in algoritms:
    results.append((  algo['name'],  RandomizedSearchCV(algo['model'],algo['params'], cv=10 ).fit(X_train,y_train)    ))
    #break

In [42]:
for result in results:
    print(result[0], result[1].best_score_, result[1].best_params_)

KNeighbors 0.567265768043901 {'weights': 'distance', 'n_neighbors': 4}
BagReg 0.8579644780759914 {'n_estimators': 16, 'base_estimator': DecisionTreeRegressor()}
Aboost 0.8648752103190478 {'n_estimators': 29, 'loss': 'exponential', 'learning_rate': 2.0, 'base_estimator': DecisionTreeRegressor()}
Gboost 0.7458463371412677 {'n_estimators': 158, 'loss': 'huber', 'learning_rate': 0.30000000000000004, 'criterion': 'mse', 'ccp_alpha': 1.0}
RandFrst 0.8297650164135346 {'n_estimators': 363, 'max_depth': 12, 'criterion': 'mse', 'ccp_alpha': 0.5}
LightGB 0.8565104854162204 {'reg_lambda': 0.4, 'reg_alpha': 0.5, 'n_estimators': 115, 'max_depth': 83, 'boosting_type': 'gbdt'}
ElastNt 0.6658485374233262 {'l1_ratio': 0.0, 'alpha': 1.0}


In [43]:
print( 'Validation Array Scores:' )
print('-'*35)

for result in results:
    y_pred = result[1].best_estimator_.predict(X_test)
    print( result[0] , ' RMSE = ', mean_squared_error(y_test,y_pred)**.5   )

Validation Array Scores:
-----------------------------------
KNeighbors  RMSE =  6.271202839294385
BagReg  RMSE =  3.3000382367950243
Aboost  RMSE =  3.6040729883373843
Gboost  RMSE =  4.363842958819417
RandFrst  RMSE =  3.4107509534354947
LightGB  RMSE =  3.528412452364783
ElastNt  RMSE =  5.054606265158046


### Вывод: 
- На train самый лучший результат показал: 
    1. Aboost (0.86)
    1. BagReg (0.8579)
    1. LightGB (0.856)
- На валидационной выборке самый лучший результат показал:  
    1. BagReg  (RMSE =  3.3)
    1. RandFrst  (RMSE =  3.41)
    1. LightGB  (RMSE =  3.52)
    
То есть, лучший результат на тестовой выборке еще не значит лучший результат на валидационной. Хотелось бы отметить BagReg на дереве показал топовый резульат на трейне и на валидации. 